## n*n-1 Puzzle problem

In [49]:
import numpy as np
import random
from icecream import ic
from enum import Enum
import queue

In [28]:
class Direction(Enum):
    UP = 1
    DOWN = 2
    RIGHT = 3
    LEFT = 4

#### Some function used to build the algorithms

In [ ]:
def create_random_matrix(n):   ##given a number n return a matrix n*n dimensional with number from 0 to n*n-1 in randomized position
    numbers = np.arange(n * n)
    np.random.shuffle(numbers)
    matrix = numbers.reshape(n, n)
    return matrix

def common_in_matrices(matrix1, matrix2): ##compare two matrices and return an array with the number that are in the same position in both two matrices
    matrix1 = np.array(matrix1)
    matrix2 = np.array(matrix2)
    comparison = matrix1 == matrix2
    result = matrix1[comparison]
    return result

def convenient_direction(n, matrix1, matrix2): #given a number return the directions to be used to move it from its position in the firtst matrix to the one in the second one
    position1 = np.where(matrix1 == n)
    position2 = np.where(matrix2 == n)
    result = list()
    if (position1[0][0]<position2[0][0]):
        result.append(Direction.DOWN)
    elif (position1[0][0]>position2[0][0]):
        result.append(Direction.UP)
    if (position1[1][0]<position2[1][0]):
        result.append(Direction.RIGHT)
    elif (position1[1][0]>position2[1][0]):
        result.append(Direction.LEFT)
    return result

def min_move(n, matrix1, matrix2):  ##given a number return the number of step that are necessary to move it from its position in the firtst matrix to the one in the second one
    direction = convenient_direction(n, matrix1, matrix2)
    position1 = np.where(matrix1 == n)
    position2 = np.where(matrix2 == n)
    result = 0
    if (Direction.UP in direction):
        result += position1[0][0]-position2[0][0]
    elif (Direction.DOWN in direction):
        result += position2[0][0]-position1[0][0]
    if (Direction.RIGHT in direction):
        result += position2[1][0]-position1[1][0]
    elif (Direction.LEFT in direction):
        result += position1[1][0]-position2[1][0]  
    return result

def get_closer(n, matrix): ##given a number return the tuples with the coordinates of all directly closed numbers in the matrix
    closer = list()
    position = np.where(matrix == n)
    if (position[0][0]>=1):
        closer.append((position[0][0]-1, position[1][0], Direction.UP))
    if (position[0][0]<=(len(matrix)-2)):
        closer.append((position[0][0]+1, position[1][0], Direction.DOWN))
    if (position[1][0]>=1):
        closer.append((position[0][0], position[1][0]-1, Direction.LEFT))
    if (position[1][0]<=(len(matrix)-2)):
        closer.append((position[0][0], position[1][0]+1, Direction.RIGHT))
    return closer


### Initialize the problem

In [66]:
SIZE = 3
START = create_random_matrix(SIZE)
GOAL = create_random_matrix(SIZE)

ic(START, GOAL)


ic| START: array([[3, 7, 6],
                  [8, 2, 0],
                  [5, 4, 1]])
    GOAL: array([[6, 1, 2],
                 [7, 8, 3],
                 [0, 4, 5]])


(array([[3, 7, 6],
        [8, 2, 0],
        [5, 4, 1]]),
 array([[6, 1, 2],
        [7, 8, 3],
        [0, 4, 5]]))

### Uniform-cost search

In [104]:

def uniformCost_algorithm(initial, GOAL):
    fronteer = queue.PriorityQueue()
    state = initial.copy()
    checked = set()
    total_moves = 0
    state_moves = 0
    checked.add(state.tobytes())
    while not np.array_equal(state, GOAL):
        total_moves += 1
        state_moves = state_moves+1
        checked.add(state.tobytes())
        closer = get_closer(0, state)
        for c in closer:
            position = np.where(state == 0)
            new_state=state.copy()
            num = state[c[0]][c[1]]
            new_state[position[0][0]][position[1][0]] = num
            new_state[c[0]][c[1]] = 0
            common = common_in_matrices(state, GOAL)
            if (new_state.tobytes() not in checked):
                if (num in common):
                    fronteer.put((3, (tuple(map(tuple, new_state)), state_moves)))
                else:
                    if (c[2] in convenient_direction(num, state, GOAL)):
                        fronteer.put((1, (tuple(map(tuple, new_state)), state_moves)))
                    else:
                        fronteer.put((2, (tuple(map(tuple, new_state)), state_moves)))
        priority, (state_as_tuple, state_moves) = fronteer.get()
        state = np.array(state_as_tuple)
    return (state_moves, total_moves)
    


### Breadth-first search

In [106]:
def breadthFirst_algorithm(initial, GOAL):
    checked = set()
    total_moves = 0
    fronteer = []
    state = initial.copy()
    state_moves = 0
    checked.add(state.tobytes())
    while not np.array_equal(state, GOAL):
        total_moves += 1
        state_moves = state_moves+1
        checked.add(state.tobytes())
        closer = get_closer(0, state)
        for c in closer:
            position = np.where(state == 0)
            new_state=state.copy()
            num = state[c[0]][c[1]]
            new_state[position[0][0]][position[1][0]] = num
            new_state[c[0]][c[1]] = 0
            if (new_state.tobytes() not in checked):
                fronteer.append((tuple(map(tuple, new_state)), state_moves)) 
        (state_as_tuple, state_moves) = fronteer.pop(0)
        state = np.array(state_as_tuple)
    return (state_moves, total_moves)

In [110]:
(this, total) = breadthFirst_algorithm(START, GOAL)
ic('Breadth-first',this,total)
(this, total) = uniformCost_algorithm(START, GOAL)
ic('Uniform-cost',this,total)
None

ic| 'Breadth-first', this: 23, total: 229998
ic| 'Uniform-cost', this: 149, total: 149919
